# Helmsman Tutorial

## Overview 

This tutorial notebook contains several demos of how to use Helmsman.



### Display helmsman's help prompt

To list all available options for Helmsman, use the following command:

In [ ]:
!python helmsman.py -h

### Download example VCFs+reference genome

For the examples provided in this tutorial, you will need to download the following files from the Google Cloud Genomics storage repository

- gzipped VCF files for Chr21 and Chr22 from the 1000 Genomes Phase 3 project
- GRCh37-lite reference genome

In [ ]:
!wget https://storage.googleapis.com/genomics-public-data/ftp-trace.ncbi.nih.gov/1000genomes/ftp/release/20130502/ALL.chr21.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz
!wget https://storage.googleapis.com/genomics-public-data/ftp-trace.ncbi.nih.gov/1000genomes/ftp/release/20130502/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz
!wget https://storage.googleapis.com/genomics-public-data/references/GRCh37lite/GRCh37-lite.fa.gz
!gunzip GRCh37-lite.fa.gz

## Run Helmsman on single VCF

The default input for Helmsman is a single VCF file. The following examples run Helmsman separately on the 1000 Genomes VCFs for Chromosome 21 and Chromosome 22. To keep the output from both chromosomes, we use the `--projectdir "1kg_chr[N]"` option.

In [ ]:
!python helmsman.py \
    --mode vcf \
    --input ALL.chr21.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz \
    --fastafile GRCh37-lite.fa \
    --projectdir "1kg_chr21" \
    --decomp pca \
    --rank 4 \
    --seed 6724420 \
    --maxac 0 \
    --verbose

In [ ]:
!python helmsman.py \
    --mode vcf \
    --input ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz \
    --fastafile GRCh37-lite.fa \
    --projectdir "1kg_chr22" \
    --decomp pca \
    --rank 4 \
    --seed 6724420 \
    --maxac 0 \
    --verbose

## Run Helmsman in a pipeline

In VCF mode, Helmsman will accept input piped from other programs by specifying `--input -`, allowing the user to perform any number of pre-processing steps to the VCF file. In the example below, we are pre-filtering the Chr22 VCF with bcftools to include only biallelic SNVs:

In [ ]:
!bcftools view -m2 -M2 -v snps ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz | \
    python helmsman.py \
        --mode vcf \
        --input - \
        --projectdir "1kg_chr22" \
        --decomp pca \
        --rank 4 \
        --seed 6724420 \
        --maxac 0 \
        --verbose

## Run Helmsman on multiple VCFs

In VCF mode, Helmsman will also accept a text file containing the file paths of the VCFs we wish to process (one per line).

This example runs with the `--cpus` option to enable parallel processing of the VCFs.

In [ ]:
# create input text file
!ls *.vcf.gz > vcfs.txt

# confirm text file has the VCFs we want
!cat vcfs.txt

# run Helmsman with VCF list as input
!python helmsman.py \
    --mode vcf \
    --input vcfs.txt \
    --fastafile GRCh37-lite.fa \
    --projectdir "1kg_combined" \
    --cpus 2

## Run Helmsman on an existing count matrix

In some cases, you may wish to re-run the mutation signature analysis using different options (e.g., with a different rank). To avoid having to re-generate the input matrix, you can use Helmsman in aggregation mode (`--mode agg`), and supply an existing matrix as the input. Here we are using the count matrix we previously generated from the Chr21 VCF.

In [ ]:
!python helmsman.py \
  --mode agg \
  --input "1kg_chr21/NMF_M_spectra.txt" \
  --verbose \
  --rank 4

## Aggregation mode with multiple input matrices

You may also encounter situations where you have generated count matrices on partitions of your data and need to recombine them. Helmsman can aggregate data in two ways:

- combining from runs on data split by genomic regions (but in identical samples)
- combining from runs on data split by different samples

### Aggregating output from different regions

If you need to combine output from multiple previous Helmsman runs on non-overlapping genomic regions, create a text file named `m_regions.txt` containing the paths to the count matrices (one per line), and use as the input file in aggregation mode (`--mode agg`).

This mode is particularly useful if you are working with very large datasets (>10,000 samples), as you can split the input into small regions and spawn many simultaneous Helmsman runs using Slurm or other workload managers. 

Note that the input matrices **must** have identical dimensions. **Samples must be ordered identically in the rows of each matrix.**

In [ ]:
# create text file containing paths of count matrices from the chr21 and chr22 output
!find . -type d -name "1kg_chr*" -exec find {} -type f -name "*spectra.txt" \; > m_regions.txt

# confirm text file has the count matrices we want
!cat m_regions.txt

# if input file is named "m_regions.txt," Helmsman will add matrices element-wise
!python helmsman.py \
  --mode agg \
  --input m_regions.txt \
  --verbose \
  --rank 4

### Aggregating output from different subsamples

In some instances, your data may be split into sub-samples--Helmsman can be run on these separately to generate the subsample count matrices, and then run again in aggregation mode to concatenate these matrices. To aggregate in this way, create a text file named `m_samples.txt` containing the paths to the subsample count matrices (one per line), and use as the input file in aggregation mode (`--mode agg`).

We recommend caution when using this mode; data should usually be split into subsamples only after variant calling has been performed over the entire sample.

In [ ]:
# create text file containing paths of count matrices from the chr21 and chr22 output
!find . -type d -name "1kg_chr*" -exec find {} -type f -name "*spectra.txt" \; > m_samples.txt

# confirm text file has the count matrices we want
!cat m_samples.txt

# if input file is named "m_samples.txt," Helmsman will concatenate matrices rowwise
# i.e., if matrix 1 is N1xK and matrix 2 is N2xK, the new matrix will be (N1+N2)xK
!python helmsman.py \
  --mode agg \
  --input m_samples.txt \
  --verbose \
  --rank 4

### Generate R script for compatibility between Helmsman and other mutation signature analysis packages

Helmsman was designed to serve as a single point-of-entry for other mutation signature analysis tools, enabling use of these tools for arbitrarily large datasets. These tools all have slightly different formatting requirements for the mutation spectra matrix, so Helmsman includes the`--package` option to generate an R script containing code necessary to coerce the output matrix into a format compatible with the specified package.

For example, suppose we want to analyze our data with the `SomaticSignatures` package. This package requires the Nx96 input matrix to be transposed, such that the mutation subtypes are the rows, and the sample IDs are the columns. Also, instead of the subtype format used by Helmsman (e.g., C_A.TCG), subtypes are formatted as "CA T.G".

The code below includes the `--package SomaticSignatures` option, which will write an R script to `1kg_chr21/Helmsman_to_SomaticSignatures.R`. Sourcing this script from within R will run the following commands:
- load the `SomaticSignatures` and `devtools` packages (assuming these have already been installed)
- install the `musigtools` package from GitHub
- read the mutation spectra matrix generated by Helmsman
- coerce the mutation spectra matrix into a format compatible with `SomaticSignatures`

In [ ]:
!python helmsman.py \
    --mode vcf \
    --input ALL.chr21.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz \
    --fastafile GRCh37-lite.fa \
    --projectdir "1kg_chr21" \
    --decomp pca \
    --rank 4 \
    --seed 6724420 \
    --maxac 0 \
    --package SomaticSignatures \
    --verbose